Extract and transform invoice data from Zoho and delta table creation

In [0]:
# Import packages

import json
import requests
#from requests_oauthlib import OAuth1
import datetime as dt
import dateutil
from pyspark.sql.functions import sum, to_timestamp, to_date, col

In [0]:
# Obtain access to Datalake en 2 Storage

spark.conf.set(
    "fs.azure.account.key.fewdatalakegen2.dfs.core.windows.net",
    dbutils.secrets.get(scope="Few-Bi-Scope", key="fewdatalakegen2key"))

In [0]:
%run "./ZohoToken"

In [0]:
zohoToken

Out[7]: '1000.2e7dee79b8192668a1466f01a4bb475b.a0aa2b0e17c4cd72a76d907a6c018375'

In [0]:
headers = {
    'Authorization': f'Zoho-oauthtoken {zohoToken}',
}
headers

Out[8]: {'Authorization': 'Zoho-oauthtoken 1000.2e7dee79b8192668a1466f01a4bb475b.a0aa2b0e17c4cd72a76d907a6c018375'}

In [0]:
type(headers)

Out[9]: dict

Out[5]: <Response [200]>

In [0]:
invoices = []

page = 1

while True:
    
    response = requests.get('https://books.zoho.com/api/v3/invoices?page={}'.format(page), headers=headers)
    res = response.json()
    for i in res['invoices']:
        invoices.append(i)
    if res['page_context']['has_more_page'] == True:
        page = +1
    else:
        break

len(invoices)

Out[10]: 87

In [0]:
invoices[0]

Out[11]: {'invoice_id': '3578547000001876083',
 'ach_payment_initiated': False,
 'zcrm_potential_id': '',
 'zcrm_potential_name': '',
 'customer_name': 'Kappa Sigma Attn. Bradley Bailey',
 'customer_id': '3578547000001412003',
 'company_name': '',
 'status': 'draft',
 'color_code': '',
 'current_sub_status_id': '',
 'current_sub_status': 'draft',
 'invoice_number': 'INV-000117',
 'reference_number': '',
 'date': '2023-02-07',
 'due_date': '2023-02-07',
 'due_days': '',
 'currency_id': '3578547000000000097',
 'schedule_time': '',
 'email': 'erios@fewinery.com',
 'currency_code': 'USD',
 'currency_symbol': '$',
 'template_type': 'standard',
 'is_viewed_by_client': False,
 'has_attachment': False,
 'client_viewed_time': '',
 'project_name': '',
 'billing_address': {'address': '',
  'street2': '',
  'city': '',
  'state': '',
  'zipcode': '',
  'country': '',
  'phone': '',
  'fax': '',
  'attention': ''},
 'shipping_address': {'address': '',
  'street2': '',
  'city': '',
  'state': '',
 

In [0]:
type(invoices[0])

Out[12]: dict

In [0]:
jsondata = json.dumps(invoices)
type(jsondata)

Out[13]: str

In [0]:
dbutils.fs.put('abfss://raw@fewdatalakegen2.dfs.core.windows.net/ZohoInvoices/ZohoInvoices.json', jsondata, True)

Wrote 171182 bytes.
Out[14]: True

In [0]:
sparkdf = spark.read.option("multiline", "true").json(sc.parallelize([jsondata]))

In [0]:
display(sparkdf)

ach_payment_initiated,adjustment,balance,billing_address,client_viewed_time,color_code,company_name,country,created_by,created_time,currency_code,currency_id,currency_symbol,current_sub_status,current_sub_status_id,custom_fields,customer_id,customer_name,date,documents,due_date,due_days,email,exchange_rate,has_attachment,invoice_id,invoice_number,invoice_url,is_emailed,is_viewed_by_client,is_viewed_in_mail,last_modified_time,last_payment_date,last_reminder_sent_date,mail_first_viewed_time,mail_last_viewed_time,payment_expected_date,phone,project_name,reference_number,reminders_sent,salesperson_id,salesperson_name,schedule_time,shipping_address,shipping_charge,status,template_id,template_type,total,transaction_type,updated_time,write_off_amount,zcrm_potential_id,zcrm_potential_name
false,0.0,11822.49,"List(, , , , , , , , )",,,,,Fairwinds,2023-02-07T11:56:55-0500,USD,3578547000000000097,$,draft,,List(),3578547000001412003,Kappa Sigma Attn. Bradley Bailey,2023-02-07,,2023-02-07,,erios@fewinery.com,1.0,false,3578547000001876083,INV-000117,null,false,false,null,2023-02-07T11:58:07-0500,,,null,null,,,,,0,3578547000000302020,Fairwinds,,"List(, , , , , , , , )",0.0,draft,3578547000001046050,standard,11822.49,renewal,2023-02-07T11:58:07-0500,0.0,,
false,0.0,1147.54,"List(, , , , , , , , )",,,"Pittsburgh Penguins, Attn Kat Smerdel",,Fairwinds,2023-01-24T16:40:45-0500,USD,3578547000000000097,$,overdue,,List(),3578547000001778077,"Pittsburgh Penguins, Attn Kat Smerdel",2023-01-24,,2023-01-24,Overdue by 15 days,ksmerdel@pittsburghpenguins.com,1.0,false,3578547000001778089,INV-000115,https://zohosecurepay.com/books/fairwindsestatewinery/secure?CInvoiceID=2-378d7d2ffa46461b6cf1769d69ff8a8544a9690a2a1296a9df13eeaccdae67ec23a50a32829160272ade26d2a3d26ca4bd12cbe4797a59056962bd3386b143baa465b6cc8b7851c7,true,false,false,2023-01-24T16:42:26-0500,,,,,,,,,0,3578547000000302020,Fairwinds,,"List(, , , , , , , , )",0.0,overdue,3578547000001125023,standard,1147.54,renewal,2023-01-24T16:42:26-0500,0.0,,
false,0.0,1152.81,"List(, , , , , , , , )",,,New Jersey Devils,,Fairwinds,2023-01-24T12:18:29-0500,USD,3578547000000000097,$,overdue,,List(),3578547000001650123,New Jersey Devils,2023-01-24,,2023-01-24,Overdue by 15 days,,1.0,false,3578547000001778037,INV-000116,https://zohosecurepay.com/books/fairwindsestatewinery/secure?CInvoiceID=2-378d7d2ffa46461b6cf1769d69ff8a8542652b1e152bd8b9df13eeaccdae67ec37283dea112cf4d22ade26d2a3d26ca4bd12cbe4797a59056962bd3386b143ba3cac44edc835bd59,true,false,false,2023-02-07T11:47:46-0500,,,,,,,,,0,3578547000000302020,Fairwinds,,"List(, , , , , , , , )",0.0,overdue,3578547000001125029,standard,1152.81,renewal,2023-02-07T11:47:46-0500,0.0,,
false,0.0,5077.61,"List(, , , , , , , , )",,,"Cincinnati Bengals, Attn. Alex Simons",,Fairwinds,2023-01-12T16:39:47-0500,USD,3578547000000000097,$,overdue,,List(),3578547000001778001,"Cincinnati Bengals, Attn. Alex Simons",2023-01-12,,2023-01-12,Overdue by 27 days,,1.0,false,3578547000001712001,INV-000114,https://zohosecurepay.com/books/fairwindsestatewinery/secure?CInvoiceID=2-378d7d2ffa46461bc8726846747ea67ba3d2dbb01bacca4bdf13eeaccdae67ec21df9fddfba3bb2e2ade26d2a3d26ca4bd12cbe4797a59056962bd3386b143ba894e1ef2054003d9,false,false,null,2023-01-24T12:15:38-0500,,,null,null,,,,,0,3578547000000302020,Fairwinds,,"List(, , , , , , , , )",1080.0,overdue,3578547000000017001,standard,5077.61,renewal,2023-01-24T12:15:38-0500,0.0,,
false,0.0,77274.11,"List(, , , , , , , , )",,,LINQTO,,Fairwinds,2023-01-10T19:06:06-0500,USD,3578547000000000097,$,overdue,,List(),3578547000001693001,LINQTO,2022-12-31,,2022-12-31,Overdue by 39 days,,1.0,false,3578547000001693038,INV-000113,https://zohosecurepay.com/books/fairwindsestatewinery/secure?CInvoiceID=2-378d7d2ffa46461b6167226f490606b983295c276207216bdf13eeaccdae67ecacd5632169624a0f2ade26d2a3d26ca4bd12cbe4797a59056962bd3386b143ba778bc4aaca2884ab,true,false,false,2023-01-26T12:08:05-0500,,,,,,,,,0,3578547000000302020,Fairwinds,,"List(, , , , , , , , )",11800.6

In [0]:
sparkdf = sparkdf.select('customer_id', 'customer_name', 'email', 'invoice_id', 'invoice_number', 'date', 'last_modified_time', 'last_payment_date', 'created_time', 'due_days', 'balance', 'shipping_charge','total', 'status')

In [0]:
display(sparkdf)

customer_id,customer_name,email,invoice_id,invoice_number,date,last_modified_time,last_payment_date,created_time,due_days,balance,shipping_charge,total,status
3578547000001412003,Kappa Sigma Attn. Bradley Bailey,erios@fewinery.com,3578547000001876083,INV-000117,2023-02-07,2023-02-07T11:58:07-0500,,2023-02-07T11:56:55-0500,,11822.49,0.0,11822.49,draft
3578547000001778077,"Pittsburgh Penguins, Attn Kat Smerdel",ksmerdel@pittsburghpenguins.com,3578547000001778089,INV-000115,2023-01-24,2023-01-24T16:42:26-0500,,2023-01-24T16:40:45-0500,Overdue by 15 days,1147.54,0.0,1147.54,overdue
3578547000001650123,New Jersey Devils,,3578547000001778037,INV-000116,2023-01-24,2023-02-07T11:47:46-0500,,2023-01-24T12:18:29-0500,Overdue by 15 days,1152.81,0.0,1152.81,overdue
3578547000001778001,"Cincinnati Bengals, Attn. Alex Simons",,3578547000001712001,INV-000114,2023-01-12,2023-01-24T12:15:38-0500,,2023-01-12T16:39:47-0500,Overdue by 27 days,5077.61,1080.0,5077.61,overdue
3578547000001693001,LINQTO,,3578547000001693038,INV-000113,2022-12-31,2023-01-26T12:08:05-0500,,2023-01-10T19:06:06-0500,Overdue by 39 days,77274.11,11800.68,77274.11,overdue
3578547000001650123,New Jersey Devils,,3578547000001650142,INV-000112,2023-01-03,2023-01-06T12:05:55-0500,,2023-01-03T20:21:48-0500,Overdue by 36 days,16553.89,3603.0,16553.89,overdue
3578547000001025175,Dan M,dan@fewinery.com,3578547000001568013,INV-000111,2022-12-23,2022-12-29T16:56:02-0500,2022-12-29,2022-12-23T16:15:12-0500,,0.0,390.0,1790.75,paid
3578547000001562001,Matt LaFleur,,3578547000001562020,INV-000110,2022-12-23,2022-12-23T17:25:34-0500,2022-12-23,2022-12-23T11:38:52-0500,,0.0,254.0,1934.9,paid
3578547000001548017,Shop the Scenes Att. Sandi Isaacs,sandiisaacs39@gmail.com,3578547000001548043,INV-000109,2022-12-22,2022-12-22T17:54:51-0500,,2022-12-22T17:28:13-0500,Overdue by 48 days,3000.38,576.0,3000.38,overdue
3578547000001025001,Luis Ramos,LUISR@FEWINERY.COM,3578547000001545155,INV-000108,2022-12-21,2022-12-21T13:09:36-0500,,2022-12-21T13:07:16-0500,,76.77,0.0,76.77,void


In [0]:
sparkdf =sparkdf.withColumn("date", to_date(col("date")))\
                .withColumn("last_modified_time", to_timestamp(col("last_modified_time")))\
                .withColumn("last_payment_date", to_date(col("last_payment_date")))\
                .withColumn("created_time", to_timestamp(col("created_time")))

In [0]:
sparkdf.agg({'total':'sum'}).show()

+----------+
|sum(total)|
+----------+
| 420581.99|
+----------+



In [0]:
sparkdf.count()

Out[21]: 87

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS zoho

In [0]:
sparkdf.write.option("overwriteSchema", "true").format("delta").mode("overwrite").saveAsTable("zoho.invoices")

In [0]:

"""
payments = []

page = 1

while True:
    
    response = requests.get('https://books.zoho.com/api/v3/invoices/{}/'.format(invoice), headers=headers)
    res = response.json()
    for i in res['invoices']:
        invoices.append(i)
    if res['page_context']['has_more_page'] == True:
        page = +1
    else:
        break

len(invoices)
"""

Out[24]: "\npayments = []\n\npage = 1\n\nwhile True:\n    \n    response = requests.get('https://books.zoho.com/api/v3/invoices/{}/'.format(invoice), headers=headers)\n    res = response.json()\n    for i in res['invoices']:\n        invoices.append(i)\n    if res['page_context']['has_more_page'] == True:\n        page = +1\n    else:\n        break\n\nlen(invoices)\n"